# Switzerland Network Analysis (Unified Pipeline)

Uses `src.analysis` + **Central Config** to calculate metrics.  
Implements **Smart Caching**: Increasing `NUM_RANDOM_SIMULATIONS` in `config.py` triggers re-runs.

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..') 
sys.path.append('.')

import pickle
import networkx as nx
from pathlib import Path
from src.analysis.metrics import NetworkAnalyzer
from src.analysis.storage import ResultsManager
from src.analysis.visualizer import NetworkVisualizer
from src.analysis.config import AnalysisConfig

In [ ]:
# CONFIG from Central File
COUNTRY = "switzerland"
GRAPH_PATH = AnalysisConfig.get_graph_path(COUNTRY)

NUM_SIMULATIONS = AnalysisConfig.NUM_RANDOM_SIMULATIONS
FRACTIONS = AnalysisConfig.FRACTIONS

print(f"Target Graph: {GRAPH_PATH}")
print(f"Random Simulations: {NUM_SIMULATIONS}")

In [ ]:
if not Path(GRAPH_PATH).exists():
     raise FileNotFoundError(f"{GRAPH_PATH} not found.")

print(f"Loading {GRAPH_PATH}...")
with open(GRAPH_PATH, 'rb') as f:
    G = pickle.load(f)
    
analyzer = NetworkAnalyzer(G)
storage = ResultsManager()
viz = NetworkVisualizer()

## 1. Global Metrics

In [ ]:
global_metrics = storage.get_cached_or_run(
    COUNTRY,
    "global_metrics",
    lambda: analyzer.calculate_global_metrics()
)
print(global_metrics)

## 2. Robustness: Efficiency Decay
Comparing Random Failures vs Targeted Attacks (Degree & Betweenness).

In [ ]:
# 2.1 Random (Efficiency)
params_random = {'num_simulations': NUM_SIMULATIONS, 'fractions': FRACTIONS}
random_eff = storage.get_cached_or_run(
    COUNTRY,
    "efficiency_decay_random",
    lambda: analyzer.simulate_random_attack_efficiency(FRACTIONS, NUM_SIMULATIONS),
    current_params=params_random
)

# 2.2 Targeted Degree
targeted_deg = storage.get_cached_or_run(
    COUNTRY,
    "efficiency_decay_degree",
    lambda: analyzer.simulate_targeted_attack(FRACTIONS, strategy='degree')
)

# 2.3 Targeted Betweenness 
# (Note: betweenness calc on large graphs is slow, smart caching helps here!)
targeted_bet = storage.get_cached_or_run(
    COUNTRY,
    "efficiency_decay_betweenness",
    lambda: analyzer.simulate_targeted_attack(FRACTIONS, strategy='betweenness')
)

# 2.4 Targeted Inverse Degree
targeted_inv_deg = storage.get_cached_or_run(
    COUNTRY,
    "efficiency_decay_inverse_degree",
    lambda: analyzer.simulate_targeted_attack(FRACTIONS, strategy='inverse_degree')
)

# 2.5 Targeted Inverse Betweenness
targeted_inv_bet = storage.get_cached_or_run(
    COUNTRY,
    "efficiency_decay_inverse_betweenness",
    lambda: analyzer.simulate_targeted_attack(FRACTIONS, strategy='inverse_betweenness')
)

# 2.6 Targeted Articulation (LCC Minimization)
targeted_art = storage.get_cached_or_run(
    COUNTRY,
    "efficiency_decay_articulation",
    lambda: analyzer.simulate_targeted_attack(FRACTIONS, strategy='articulation')
)

In [ ]:
# Visualization
viz.plot_efficiency_decay({
    'Random Failure': random_eff,
    'Targeted (Degree)': targeted_deg,
    'Targeted (Betweenness)': targeted_bet,
    'Inverse Targeted (Degree)': targeted_inv_deg,
    'Inverse Targeted (Betweenness)': targeted_inv_bet,
    'Targeted (Articulation)': targeted_art
}, title=f"Network Efficiency Decay - {COUNTRY.title()}")

In [ ]:
# Map visual (Interactive)
viz.create_interactive_map_ui(G)